In [1]:
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
st. set_page_config(layout="wide")
#Description
#Open dataset

dataset = 'B:/Documenten/TU_Delft/Master/Module_1/Python/Assignment/TIL6022/TILL6022_Emission_Dataset.csv'
bounding_boxes = 'B:/Documenten/TU_Delft/Master/Module_1/Python/Assignment/TIL6022/country_bb.csv'
df = pd.read_csv(dataset, delimiter=',', encoding='ISO-8859-1')
country_bb = pd.read_csv(bounding_boxes, delimiter=',', encoding='ISO-8859-1')

# Side bar code
sidebar = st.sidebar
sidebar.title('Carbon Emissions in Transport')
given_time = sidebar.selectbox('Select time period to visualise', ['Day','Week','Month','Quartile','Year'])

if given_time == 'Day':
    date_filt = 'D'
elif given_time == 'Week':
    date_filt = 'W-MON'
elif given_time == 'Month':
    date_filt = 'MS'
elif given_time == 'Quartile':
    date_filt = 'QS'
elif given_time == 'Year':
    date_filt = 'Y'
    
sidebar.write('Test 123')

default_country = int(np.where(df['country'].unique() == 'WORLD')[0])
given_country = sidebar.selectbox('Select country to visualise', df.country.unique(), index = default_country)

default_sector = int(np.where(df['sector'].unique() == 'Total')[0])
given_sector = sidebar.selectbox('Select sector to visualise', df.sector.unique(), index = default_sector)


missing_EU = ['Belgium','Finland','Estonia','Austria','Luxembourg','Greece','Malta','Netherlands','Hungary','Bulgaria','Latvia','Lithuania','Slovenia','Croatia','Ireland','Portugal','Slovakia','Denmark','Polan']
df.date = pd.to_datetime(df['date'])
df_filt = df.groupby(['country','sector']).resample(date_filt,on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
df_filt = df_filt.reset_index()
box_pts = 4
box = np.ones(box_pts)/box_pts
df_filt["co2_smooth"] = np.convolve(df_filt.co2, box, mode='same')

df_filt = df_filt.set_index('date')
date_min = df_filt.index.min()
date_max = df_filt.index.max()
index_min = df_filt.index == date_min
index_max = df_filt.index == date_max
df_filt = df_filt.reset_index()

df_filt.date = np.datetime_as_string(df_filt.date, unit='D')
index_min_1 = pd.Series(df_filt.index[index_min])
index_min_2 = pd.Series(df_filt.index[index_min]+1)
index_max_1 = pd.Series(df_filt.index[index_max])
index_max_2 = pd.Series(df_filt.index[index_max]-1)
index_series = pd.Index(index_max_1.append([index_max_2,index_min_1,index_min_2], ignore_index=True))
df_filt.drop(index_series, axis=0, inplace= True)
zoom_factor = 3
#Drop down menus

given_country = 'UK'
if (given_country == 'WORLD' or given_country == 'ROW' or given_country == 'EU27 & UK'):
    latrange = [-90,90]
    lonrange = [-180,180]
elif given_country == 'US':
    given_country = 'United States'
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin'])-zoom_factor,float(given_country_bb['latmax'])+zoom_factor]
    lonrange = [float(given_country_bb['lonmin'])-zoom_factor*2,float(given_country_bb['lonmax'])+zoom_factor*2]
elif given_country == 'UK':
    given_country = 'United Kingdom'
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin'])-zoom_factor,float(given_country_bb['latmax'])+zoom_factor]
    lonrange = [float(given_country_bb['lonmin'])-zoom_factor*2,float(given_country_bb['lonmax'])+zoom_factor*2]
else:
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin'])-zoom_factor,float(given_country_bb['latmax'])+zoom_factor]
    lonrange = [float(given_country_bb['lonmin'])-zoom_factor*5,float(given_country_bb['lonmax'])+zoom_factor*5]

data_sector = df_filt[df_filt.sector == given_sector]
if given_country == 'WORLD':
    leg_min = float(data_sector[data_sector.country != 'WORLD']['co2'].min())
    leg_max = float(data_sector[data_sector.country != 'WORLD']['co2'].max())
elif len(df_filt[df_filt.sector == given_sector].country.unique())>1:
    leg_min = float(data_sector[data_sector.country == given_country]['co2'].min())
    leg_max = float(data_sector[data_sector.country == given_country]['co2'].max())


# filter dataset on Transport sectors: 'International Shipping' or 'International Aviation' or 'Domestic Aviation' or 'Ground Transport'
transport_data =  df_filt[df_filt.sector.isin(['International Shipping', 'International Aviation', 'Domestic Aviation', 'Ground Transport'])]
transport_data[["year", "month", "day"]] = transport_data["date"].str.split("-", expand = True)
transport_data['same_year'] = '2019' + '-' + transport_data['month'].astype(str) + '-' + transport_data['day'] # Same year is created to visualize in same plot

# add trace
Years = transport_data.year.unique()
Countries = transport_data.country.unique()


### Figures

fig_world = px.choropleth(df_filt[df_filt.sector == given_sector], 
                    # geojson= counties,
                    # featureidkey='ISO_A3',
                    locations= 'country',
                    locationmode= "country names",
                    color='co2',
                    range_color= [leg_min, leg_max],
                    #hover_name="country",
                    animation_frame= 'date'
)
fig_world.update_geos(lataxis_range=latrange, lonaxis_range=lonrange)  
fig_world.update_layout(
    title_text = "CO2 emmissions transport sector",
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection_type = "equirectangular",
    ),
    height = 600,
    width = 1200,
    
    margin=dict(l=0, r=0, t=0, b=0)

)

fig_transport = make_subplots(rows = 1, 
                    cols = 4,
                    subplot_titles = ('Total', 'International Aviation', 'Domestic Aviation', 'Ground Transport'))
trans_tot = transport_data[(transport_data.country == given_country)].groupby(
        ['date'],as_index = False).agg({'country':'first','co2':'sum','co2_smooth':'sum','year':'first','month':'first','day':'first','same_year':'first'})
trans_int_av = transport_data[(transport_data.country == given_country)
                & (transport_data.sector == 'International Aviation')]
trans_do_av = transport_data[(transport_data.country == given_country)
                & (transport_data.sector == 'Domestic Aviation')]
trans_gr_tr = transport_data[(transport_data.country == given_country)
                & (transport_data.sector == 'Ground Transport')]
for i in Years:
        lop = fig_transport.add_traces(
                [
                        go.Scatter(x=list(trans_tot[(trans_tot.year == i)].same_year),
                                y=list(trans_tot[(trans_tot.year == i)].co2),
                                name= 'Total' + given_country + i,
                                visible = True), 
                        go.Scatter(x=list(trans_int_av[(trans_int_av.year == i)].same_year),
                                y=list(trans_int_av[(trans_int_av.year == i)].co2),
                                name= 'International Aviation' + given_country + i,
                                visible = True),
                        go.Scatter(x=list(trans_do_av[(trans_do_av.year == i)].same_year),
                                y=list(trans_do_av[(trans_do_av.year == i)].co2),
                                name= 'Domestic Aviation' + given_country + i,
                                visible = True),
                        go.Scatter(x=list(trans_gr_tr[(trans_gr_tr.year == i)].same_year),
                                y=list(trans_gr_tr[(trans_gr_tr.year == i)].co2),
                                name= 'Ground Transport' + given_country + i,
                                visible = True)
                ], rows=[1,1,1,1], cols=[1,2,3,4]
        )

fig_transport.update_layout(
    height = 500,
    width = 1800
)

fig_transport.update_layout(xaxis_title = 'Date', yaxis_title = 'CO2 emissions x 1000 (ppm)')
fig_transport.update_layout(title_text = 'CO2 emissions comparison 2019 until 2022')
fig_transport.update_layout(xaxis=dict(tickformat="%d-%B"),
                 xaxis2=dict(tickformat="%d-%B"),
                 xaxis3=dict(tickformat="%d-%B"),
                 xaxis4=dict(tickformat="%d-%B")
                 )
fig_transport['layout']['yaxis2']['title']='CO2 emissions x 1000 (ppm)'
fig_transport['layout']['yaxis3']['title']='CO2 emissions x 1000 (ppm)'
fig_transport['layout']['yaxis4']['title']='CO2 emissions x 1000 (ppm)'
fig_transport['layout']['xaxis2']['title']='Date'
fig_transport['layout']['xaxis3']['title']='Date'
fig_transport['layout']['xaxis4']['title']='Date'

Delta_Country = []
Date_country = []
Date_country_max = []
Percentage_Delta = []
Delta_Country_Do = []
Date_country_Do = []
Date_country_max_Do = []
Percentage_Delta_Do = []
Delta_Country_Trans = []
Date_country_Trans = []
Date_country_max_Trans = []
Percentage_Delta_Trans = []
Delta_Country_Int = []
Date_country_Int = []
Date_country_max_Int = []
Percentage_Delta_Int = []
# Total sectors deviation
Delta_Total = trans_tot[(trans_tot.country == given_country)]
print(Delta_Total)

diff_list = []
for d in range(len(Delta_Total)-1):
        Diff = abs(Delta_Total['co2_smooth'].iloc[d] - Delta_Total['co2_smooth'].iloc[d+1])
        Co2Value = Delta_Total['co2_smooth'].iloc[d-1]
        diff_list.append(Diff)
        Date_country.append(Delta_Total.iloc[d]['date'])
Max_Diff = max(diff_list)
percent_Delta = ("%g" % round((Max_Diff/Co2Value)*100,1))
index_maxvalue = diff_list.index(Max_Diff)
maxDATE = Date_country[index_maxvalue]
Delta_Country.append(Max_Diff)
Date_country_max.append(maxDATE)
Percentage_Delta.append(percent_Delta)

# Domestic sector 
Do_Av = trans_do_av[(trans_do_av.country == given_country)]
diff_list_Do = []
for d in range(len(Do_Av)-1):
        Diff = abs(Do_Av['co2_smooth'].iloc[d] - Do_Av['co2_smooth'].iloc[d+1])
        Co2Value_Do = Do_Av['co2_smooth'].iloc[d-1]
        diff_list_Do.append(Diff)
        Date_country_Do.append(Do_Av.iloc[d]['date'])
Max_Diff_Do = max(diff_list_Do)
percent_Delta_Do = round((Max_Diff_Do/Co2Value_Do)*100,1)
index_maxvalue_Do = diff_list_Do.index(Max_Diff_Do)
maxDATE_Do = Date_country_Do[index_maxvalue_Do]
Delta_Country_Do.append(Max_Diff_Do)
Date_country_max_Do.append(maxDATE_Do)
Percentage_Delta_Do.append(percent_Delta_Do)

Gr_Trans = trans_gr_tr[(trans_gr_tr.country == given_country)]
diff_list_Trans = []
for d in range(len(Gr_Trans)-1):
        Diff = abs(Gr_Trans['co2_smooth'].iloc[d] - Gr_Trans['co2_smooth'].iloc[d+1])
        Co2Value_Trans = Gr_Trans['co2_smooth'].iloc[d-1]
        diff_list_Trans.append(Diff)
        Date_country_Trans.append(Gr_Trans.iloc[d]['date'])
Max_Diff_Trans = max(diff_list_Trans)
Percent_Delta_Trans = round((Max_Diff_Trans/Co2Value_Trans)*100, 1)
# print('This is the max diff:' , Max_Diff_Trans, k)
Delta_Country_Trans.append(Max_Diff_Trans)
index_maxvalue_Trans = diff_list_Trans.index(Max_Diff_Trans)
maxDATE_Trans = Date_country_Trans[index_maxvalue_Trans]
Date_country_max_Trans.append(maxDATE_Trans)
Percentage_Delta_Trans.append(Percent_Delta_Trans)
# print(Delta_Country)

# for i in Years:
Int_Av = trans_int_av[(trans_int_av.country == given_country)]
diff_list_Int = []
for d in range(len(Int_Av)-1):
        Diff = abs(Int_Av['co2_smooth'].iloc[d] - Int_Av['co2_smooth'].iloc[d+1])
        Co2Value_Int = Int_Av['co2_smooth'].iloc[d-1]
        diff_list_Int.append(Diff)
        Date_country_Int.append(Int_Av.iloc[d]['date'])
Max_Diff_Int = max(diff_list_Int)
Percent_Delta_Int = round((Max_Diff_Int/Co2Value_Int)*100, 1)
Delta_Country_Int.append(Max_Diff_Int)
index_maxvalue_Int = diff_list_Int.index(Max_Diff_Int)
maxDATE_Int = Date_country_Int[index_maxvalue_Int]
Date_country_max_Int.append(maxDATE_Int)
Percentage_Delta_Int.append(Percent_Delta_Int)

DeltaTable = pd.DataFrame(
    {'Date Total': Date_country_max,
     'Total Delta': Delta_Country,
     'Percentage Total [%]': Percentage_Delta,
     'Date Ground Transportation': Date_country_max_Trans,
     'Ground Transportation Delta': Delta_Country_Trans,
     'Percentage Ground Transportation [%]': Percentage_Delta_Trans,
     'Domestic Aviation Delta': Delta_Country_Do,
     'Percentage Domestic Aviation [%]': Percentage_Delta_Do,
     'Date Domestic Aviation': Date_country_max_Do,
     'International Aviation Delta': Delta_Country_Int,
     'Percentage International Aviation [%]': Percentage_Delta_Int,
     'Date International Aviation': Date_country_max_Int
    })
DeltaTrans = DeltaTable.transpose()
DeltaTrans.style.background_gradient(cmap='Blues')
DeltaTrans.columns = [given_country]
with st.sidebar:
    st.table(DeltaTrans)
with st.container():
    st.plotly_chart(fig_world, use_container_width=True)
    st.plotly_chart(fig_transport, use_container_width=True)


2022-11-11 02:01:25.443 
  command:

    streamlit run c:\Users\timtr\anaconda3\envs\TIL6022\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
C:\Users\timtr\AppData\Local\Temp\ipykernel_20360\446608289.py:43: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_filt = df.groupby(['country','sector']).resample(date_filt,on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
C:\Users\timtr\AppData\Local\Temp\ipykernel_20360\446608289.py:43: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_filt = df.groupby(['country','sector']).resample(date_filt,on='date').sum()    #D for Day, MS for month, QS 

            date country           co2    co2_smooth  year month day  \
0     2019-01-03   WORLD  21872.218968  20523.149157  2019    01  03   
1     2019-01-04   WORLD  22058.583048  21192.313055  2019    01  04   
2     2019-01-05   WORLD  20675.199425  20786.042250  2019    01  05   
3     2019-01-06   WORLD  18538.167557  20700.506353  2019    01  06   
4     2019-01-07   WORLD  21530.075382  20730.125248  2019    01  07   
...          ...     ...           ...           ...   ...   ...  ..   
1238  2022-05-25   WORLD  24564.898052  23917.015871  2022    05  25   
1239  2022-05-26   WORLD  23818.142561  24115.490650  2022    05  26   
1240  2022-05-27   WORLD  23965.442830  23499.467755  2022    05  27   
1241  2022-05-28   WORLD  21649.387576  22255.420629  2022    05  28   
1242  2022-05-29   WORLD  19588.709550  21496.490594  2022    05  29   

       same_year  
0     2019-01-03  
1     2019-01-04  
2     2019-01-05  
3     2019-01-06  
4     2019-01-07  
...          ...  
12

Useful commands for geopandas
#ar = gdf[gdf.ADMIN == 'Aruba'] #To find the country Aruba
#print(ar.area)                 #Print area of country Aruba
# gdf.ADMIN['Aruba']
#counties['features'][3]['geometry'] access coordinates and type

In [54]:
# with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries.geojson") as f:
#     counties = geojson.load(f)
# gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)
# mask = gdf.area >2    # Function to remove smaller countries to speed up 
# gdf_selec = gdf.loc[mask]
# gdf_selec.head()
# ar = gdf[gdf.ADMIN == 'China']['geometry'] # access geometry data of China

#counties['features'][3]['geometry'] access coordinates and type
# counties['features']['properties']
# with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries_simp.geojson") as f:
#     counties = json.load(f)
# with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries_simp.geojson") as f:
#     counties = json.load(f)
# gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)
# counties['features'][2]['properties']


TypeError: list indices must be integers or slices, not str

In [55]:
# dept_geo = [0]*len(counties['features'])

# Get data from dataset (using json.loads to transform string into dict for geoshape column)
# cd_depts = gdf.ADMIN.tolist()
# dept_geo[0] = counties['features'][0]['geometry']['coordinates']
#counties['features'][0]

{'type': 'Feature',
 'geometry': None,
 'properties': {'ADMIN': 'Aruba', 'ISO_A3': 'ABW'}}

In [40]:
# shapes = [0]*len(counties['features'])
# for feature in range(len(counties['features'])):
#   shapes[feature] = counties['features'][feature]['geometry']['coordinates']
#   # for cood in range(len(feature['geometry']['coordinates'])):
#     # shapes[cood] = feature['geometry'][cood]['coordinates']
# cd_depts = gdf.ADMIN.tolist()
# for cd_dept, shape in zip(cd_depts, shapes):
    
#     # Mapping information from data to GeoJSON file
#     dept_geo.append({
#         'type': 'Feature',
#         'geometry': shape,
#         'id':cd_dept
#     })

# # Encapsulate dept_geo in a real GeoJSON formatted file
# dept_geo_ok = {'type': 'FeatureCollection', 'features': dept_geo}

In [2]:
# import pandas as pd
# import plotly
# import numpy as np


# from urllib.request import urlopen
# import json
# with urlopen('https://opendata.arcgis.com/datasets/c572ffec290b42768f80e43d31ac53a6_0.geojson') as response:
#     counties = json.load(response)
    
# df = pd.read_excel('testdata.xlsx',dtype={"cty19cd": str})
# df.columns=['CODE','NAME','VALUE']

# import plotly.express as px

# fig = px.choropleth(df, geojson=counties, locations='CODE', featureidkey="properties.cty19cd", color='VALUE',
#                            color_continuous_scale="Inferno",
#                             scope="europe")

# fig.update_geos( fitbounds="locations", visible=True)
# fig.write_html('maptest.html',auto_open=True)
missing_EU = ['Belgium','Finland','Estonia','Austria','Luxembourg','Greece','Malta','Netherlands','Hungary','Bulgaria','Latvia','Lithuania','Slovenia','Croatia','Ireland','Portugal','Slovakia','Denmark','Polan']

array(['Brazil', 'China', 'EU27 & UK', 'France', 'Germany', 'India',
       'Italy', 'Japan', 'ROW', 'Russia', 'Spain', 'UK', 'US', 'WORLD'],
      dtype=object)

In [25]:
# map = folium.Map(
#     location=[
#         gdf_selec.geometry.bounds.miny.mean(),
#         gdf_selec.geometry.bounds.minx.mean()
#     ],
#     zoom_start=1
# )

# gjson = folium.features.GeoJson(
#     gdf_selec,
# ).add_to(map)
# map

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo = []

# Get data from dataset (using json.loads to transform string into dict for geoshape column)
cd_depts = gdf.ADMIN.tolist()
shapes = gdf.geometry
cd = mapping(shapes)
# # Iterate over lines
for cd_dept, shape in zip(cd_depts, shapes):
    
    # Mapping information from data to GeoJSON file
    dept_geo.append({
        'type': 'Feature',
        'geometry': shape,
        'id':cd_dept
    })

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo_ok = {'type': 'FeatureCollection', 'features': dept_geo}


NameError: name 'gdf' is not defined

In [5]:
# fig = px.choropleth(vf, geojson=gdf_selec, color="co2",
#                     locations="ISO-A3", featureidkey="properties.district",
#                     projection="mercator"
#                    )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
given_time = st.selectbox('Select time period to visualise', ['Day','Month','Quartile','Year'])
if given_time == 'Day':
    date_filt = 'D'
elif given_time == 'Month':
    date_filt = 'MS'
elif given_time == 'Quartile':
    date_filt = 'QS'
elif given_time == 'Year':
    date_filt = 'Y'

given_sector = st.selectbox('Select sector to visualise', df_filt.sector.unique())

default_country = int(np.where(df['country'].unique() == 'WORLD')[0])
given_country = st.selectbox('Select country to visualise', df_filt.country.unique(), index = default_country)
default_sector = int(np.where(df['sector'].unique() == 'Total')[0])
given_sector = st.selectbox('Select sector to visualise', df_filt.sector.unique(), index = default_sector)
# given_country = "China"
# country_bb.country.index(given_country)
# country_bb.country[(country_bb.country.values==given_country).any(0)]

given_sector = 'International Shipping'
given_country = 'US'
if (given_country == 'WORLD' or given_country == 'ROW' or given_country == 'EU27 & UK'):
    latrange = [-90,90]
    lonrange = [-180,180]
elif given_country == 'US':
    given_country = 'United States'
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin']),float(given_country_bb['latmax'])]
    lonrange = [float(given_country_bb['lonmin']),float(given_country_bb['lonmax'])]
else:
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin']),float(given_country_bb['latmax'])]
    lonrange = [float(given_country_bb['lonmin']),float(given_country_bb['lonmax'])]

data_sector = df_filt[df_filt.sector == given_sector]
if given_country == 'WORLD':
    leg_min = float(data_sector[data_sector.country != 'WORLD']['co2'].min())
    leg_max = float(data_sector[data_sector.country != 'WORLD']['co2'].max())
elif len(df_filt[df_filt.sector == given_sector].country.unique())>1:
    leg_min = float(data_sector[data_sector.country == given_country]['co2'].min())
    leg_max = float(data_sector[data_sector.country == given_country]['co2'].max())


2022-10-18 15:12:10.687 
  command:

    streamlit run c:\Users\timtr\anaconda3\envs\TIL6022\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


Write something about the zoom function and the implemenetation of geojson files and why they are not used.

In [7]:
given_time = 'MS'
df.date = pd.to_datetime(df.date)
df_filt = df.groupby(['country','sector']).resample(given_time,on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
df_filt = df_filt.reset_index()
df_filt.date = np.datetime_as_string(df_filt.date, unit='D')
df_filt.dtypes
# df_filtdf_filt[df_filt.country=='EU27 & UK'] 

,country,sector,date,co2
574,EU27 & UK,Domestic Aviation,2019-01-01,1242.040164
575,EU27 & UK,Domestic Aviation,2019-02-01,1176.891994
576,EU27 & UK,Domestic Aviation,2019-03-01,1337.273622
577,EU27 & UK,Domestic Aviation,2019-04-01,1368.893036
578,EU27 & UK,Domestic Aviation,2019-05-01,1456.900692
...,...,...,...,...
856,EU27 & UK,Total,2022-01-01,303709.090297
857,EU27 & UK,Total,2022-02-01,264902.114454
858,EU27 & UK,Total,2022-03-01,293708.139097
859,EU27 & UK,Total,2022-04-01,251137.399223


In [26]:

fig = px.choropleth(df_filt[df_filt.sector == given_sector], 
                    # geojson= counties,
                    # featureidkey='ISO_A3',
                    locations= 'country',
                    locationmode= "country names",
                    color='co2',
                    range_color= [leg_min, leg_max],
                    #hover_name="country",
                    animation_frame= 'date'
                    #zoom=2,
)
fig.update_geos(lataxis_range=latrange, lonaxis_range=lonrange)  
fig.update_layout(
    title_text = "CO2 emmissions transport sector",
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection_type = "equirectangular",
    ),
    
                    #color_continuous_scale="Blues_r
    height=1000,
    width  = 1080
    #margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()

In [4]:
# filter dataset on Transport sectors: 'International Shipping' or 'International Aviation' or 'Domestic Aviation' or 'Ground Transport'
transport_datas = ['International Shipping', 'International Aviation', 'Domestic Aviation', 'Ground Transport']
transport_data =  df_filt[df_filt.sector.isin(transport_datas)]

# Transport_not_World = transport_data[transport_data.country != 'WORLD'].groupby(['country', 'date']).sum()

#reset index for optimal dataset and split date into years

# transport_data = Transport_not_World.reset_index(['country', 'date'])
transport_data[["year", "month", "day"]] = transport_data["date"].str.split("-", expand = True)
transport_data['same_year'] = '2019' + '-' + transport_data['month'].astype(str) + '-' + transport_data['day'] # Same year is created to visualize in same plot
transport_data2 = transport_data[(transport_data.year == '2019') | (transport_data.year == '2020')]
# transport_data["date"].str.split("-", expand = True)
fig_transport = go.Figure()
# add trace
Years = ['2019', '2020']
Countries = ['UK', 'US', 'China', 'EU27 & UK', 'France', 'Germany', 'India', 'Italy', 'Japan', 'Russia', 'Spain']
months = list(transport_data.month.unique())


list_updatemenus = []

for n, country in enumerate(Countries):
    visible = [False] * 2 * len(Countries)
    visible[2*n] = True
    visible[2*n + 1] = True
    temp_dict = dict(label = str(country),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': ('CO2 emissions comparison 2019 versus 2020 in %s'% country)}])
    list_updatemenus.append(temp_dict)

list_menu = pd.DataFrame(list_updatemenus)





C:\Users\timtr\AppData\Local\Temp\ipykernel_24036\748041950.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_data[["year", "month", "day"]] = transport_data["date"].str.split("-", expand = True)
C:\Users\timtr\AppData\Local\Temp\ipykernel_24036\748041950.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_data[["year", "month", "day"]] = transport_data["date"].str.split("-", expand = True)
C:\Users\timtr\AppData\Local\Temp\ipykernel_24036\748041950.py:10: SettingWithCopyWarning: 
A 

In [13]:

fig_transport = make_subplots(rows = 4, 
                    cols = 1,
                    subplot_titles = ('Total', 'International Aviation', 'Domestic Aviation', 'Ground Transport'))
given_country = 'WORLD'
for i in Years:
        z = transport_data[(transport_data.country == given_country) & (transport_data.year == i)]
        Int_Av = transport_data2[(transport_data2.country == given_country) & (transport_data2.year == i) & (transport_data2.sector == 'International Aviation')]
        Do_Av = transport_data2[(transport_data2.country == given_country) & (transport_data2.year == i) & (transport_data2.sector == 'Domestic Aviation')]
        Gr_Trans = transport_data2[(transport_data2.country == given_country) & (transport_data2.year == i) & (transport_data2.sector == 'Ground Transport')]
        lop = fig_transport.add_traces(
                [
                    go.Scatter(x=list(z.same_year),
                        y=list(z.co2),
                        name= given_country + i,
                        visible = True),        
                    go.Scatter(x=list(Int_Av.same_year),
                            y=list(Int_Av.co2),
                            name= 'International Aviation' + given_country + i,
                            visible = True),
                    go.Scatter(x=list(Do_Av.same_year),
                            y=list(Do_Av.co2),
                            name= 'Domestic Aviation' + given_country + i,
                            visible = True),
                    go.Scatter(x=list(Gr_Trans.same_year),
                            y=list(Gr_Trans.co2),
                            name= 'Ground Transport' + given_country + i,
                            visible = True)
                ], rows=list(range(1,5)), cols=[1,1,1,1]
                )

fig_transport.update_layout(
    height = 1500
)

fig_transport.update_layout(xaxis_title = 'Date', yaxis_title = 'CO2 emissions x 1000 (ppm)')
fig_transport.update_layout(title_text = 'CO2 emissions comparison 2019 versus 2020')
fig_transport.update_layout(xaxis=dict(tickformat="%d-%B"),
                 xaxis2=dict(tickformat="%d-%B"),
                 xaxis3=dict(tickformat="%d-%B"),
                 xaxis4=dict(tickformat="%d-%B")
                 )
fig_transport['layout']['yaxis2']['title']='CO2 emissions x 1000 (ppm)'
fig_transport['layout']['yaxis3']['title']='CO2 emissions x 1000 (ppm)'
fig_transport['layout']['yaxis4']['title']='CO2 emissions x 1000 (ppm)'
fig_transport['layout']['xaxis2']['title']='Date'
fig_transport['layout']['xaxis3']['title']='Date'
fig_transport['layout']['xaxis4']['title']='Date'


fig_transport.show()
